# 단어 임베딩 
## 코사인 유사도 
- 벡터 공간에서 유사도를 나타내는 방법 
- 원핫 인코딩으로 표현한 두 벡터의 코사인 유사도는 항상 0
- --> 밀집 벡터 필요 --> "임베딩 기법"
- 모든 실수 차원으로 원하는 차원 수만큼 단어 표현

In [1]:
import torch

x1 = torch.FloatTensor([1,2,3,4])
x2 = torch.FloatTensor([1,4,2,1])

print(torch.cosine_similarity(x1, x2, dim=0))

tensor(0.7396)


In [2]:
# 원핫인코딩으로 코사인 유사도 계산하면 안되는 이유
word1 = torch.FloatTensor([0,1,0,0,0])
word2 = torch.FloatTensor([0,0,0,1,0])

print(torch.cosine_similarity(word1, word2, dim=0))

tensor(0.)


## 임베딩 조회 (embedding look up)
- "행렬 곱셈": 원핫인코딩된 단어들과 임베딩 행렬 간의 곱셈
- --> 오차 역전파 가능--> 임베딩 표현된 문장을 목적에 맞게 훈련 가능
- torch.nn.Embedding 으로 임베딩 층 구현 가능
- 임베딩 층의 입력은 무조건 LongTensor 타입 인덱스여야 함. 

In [3]:
tokens = "We are going to watch Avengers End Game".split()
new_sent = "We are Avengers".split()
vocab = {token:i for i,token in enumerate(tokens)}

# 임베딩 층 사용하지 않을 경우 임베딩 기법 구현
embedding = torch.FloatTensor([[1,5,7],
                              [2,1,8],
                              [1,4,5],
                              [4,1,1],
                              [1,8,9],
                              [6,1,10],
                              [3,2,2],
                              [1,5,4]])

idxes = torch.LongTensor([vocab[token] for token in new_sent])
print(embedding[idxes, :])
print()

# 파이토치에서 사용하는 방법
import torch.nn as nn

embedding_layer = nn.Embedding(num_embeddings=len(vocab),
                              embedding_dim=3,
                              _weight=embedding)

# 해당하는 index 조회
print(embedding_layer(idxes))

tensor([[ 1.,  5.,  7.],
        [ 2.,  1.,  8.],
        [ 6.,  1., 10.]])

tensor([[ 1.,  5.,  7.],
        [ 2.,  1.,  8.],
        [ 6.,  1., 10.]], grad_fn=<EmbeddingBackward>)


## Gensim

In [2]:
from konlpy.tag import Mecab
from gensim.models.word2vec import Word2Vec

# 품사정보를 "(단어)/(품사정보)" 처럼 함께 저장하기위해 tokenizer 함수를 정의
mecab = Mecab('C:/mecab/mecab-ko-dic')
tokenizer = lambda x: ['/'.join((token.lower(), pos.lower())) for (token, pos) in mecab.pos(x)]

with open("./data/nsmc/ratings.txt", encoding='utf8') as file:
    # 행단위로 데이터 분리. 첫 행은 header 제외
    raw_data = file.read().splitlines()[1:]
    # 텍스트 데이터만 사용 
    data = [line.split('\t')[1] for line in raw_data]
    # 토큰화 진행 
    data = [tokenizer(sent) for sent in data]
    
model = Word2Vec(sentences=data, vector_size=100, window=5, min_count=3, sg=1)

# 훈련 완료 후 불필요한 메모리 제거
model.init_sims(replace=True)

# 단어 임베딩 행렬의 크기
print(model.wv.vectors.shape)

# 모델 저장: 파일의 첫번째 줄에는 임베딩 행렬의 크기가 적혀있다.
model.wv.save_word2vec_format('./word2vec.pt')

C:\Users\answl\anaconda3\envs\nlp\lib\site-packages\ipykernel_launcher.py:19: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).


(30382, 100)


In [9]:
len(data)

200000

In [10]:
model

In [11]:
model.wv

In [12]:
model.wv.vectors

array([[-4.22248952e-02,  9.41601768e-02,  4.21626261e-03, ...,
        -1.38073772e-01,  2.94470266e-02,  1.48032997e-02],
       [-5.42283878e-02,  7.47420564e-02, -9.44374874e-03, ...,
        -1.64438054e-01, -1.51937391e-04, -6.72569079e-03],
       [ 5.28738229e-03,  9.87587348e-02, -1.14748582e-01, ...,
        -7.00663701e-02,  4.04933915e-02, -6.67541921e-02],
       ...,
       [-1.08063646e-01,  1.15178891e-01,  8.33605900e-02, ...,
        -1.27238229e-01, -1.66832078e-02,  7.10420385e-02],
       [-3.18642035e-02,  1.09030612e-01,  1.88518986e-02, ...,
        -1.64858624e-01, -9.71087217e-02,  1.16480373e-01],
       [-1.00412704e-02,  2.07997292e-01,  4.18835543e-02, ...,
        -1.76711723e-01, -5.25666922e-02,  1.60133615e-01]], dtype=float32)

## 단어 간 유사도 구하기

In [3]:
# 1. '여배우'와 '배우'의 유사도
sim1 = model.wv.similarity(*tokenizer("여배우 배우"))
print("similarity(여배우, 배우) = {:.2f}".format(sim1))

similarity(여배우, 배우) = 0.79


In [4]:
# 2. '스토리'와 가장 유사한 단어 Top 5
sim2 = model.wv.most_similar(tokenizer('스토리'), topn=5)

for t,s in sim2:
    print("{} = {:.2f}".format(t,s))

전개/nng = 0.83
시나리오/nng = 0.82
내용/nng = 0.81
줄거리/nng = 0.78
설정/nng = 0.77


In [5]:
# 3. 벡터 연산: '남자배우' - '남자' = '연기자'
sim3 = model.wv.most_similar(positive=tokenizer('남자배우'),
                            negative=tokenizer('남자'),
                            topn = 1)
print(sim3)

[('여배우/nng', 0.7906150817871094)]


In [6]:
sim3 = model.wv.most_similar(positive=tokenizer('남자배우'),
                            negative=tokenizer('남자'),
                            topn = 5)
print(sim3)

[('여배우/nng', 0.7906150817871094), ('출연진/nng', 0.7714549899101257), ('연기자/nng', 0.7703227400779724), ('조연/nng', 0.7573984265327454), ('명배우/nng', 0.7529410123825073)]


In [8]:
import gensim
import torch
import torch.nn as nn
from torchtext.legacy.data import Field
from torchtext.legacy.data import TabularDataset
from torchtext.legacy.data import Iterator
from torchtext.vocab import Vectors
from konlpy.tag import Mecab

mecab = Mecab('C:/mecab/mecab-ko-dic')
tokenizer = lambda x: ['/'.join((token.lower(), pos.lower()))
                      for token, pos in mecab.pos(x)]

# 필드 정의
TEXT = Field(sequential=True,
            use_vocab=True,
            tokenize=tokenizer,
            lower=True,
            batch_first=True)

LABEL = Field(sequential=True,
             use_vocab=False,
             preprocessing=lambda x: int(*x),
             batch_first=True,
             is_target=True)

ID = Field(sequential=False,
          use_vocab=False,
          is_target=False)

dataset = TabularDataset(path='./data/nsmc/ratings.txt',
                        format='tsv',
                        fields=[('id', ID), ('text', TEXT), ('label', LABEL)],
                        skip_header=True)

# 훈련된 임베딩 행렬 사용 위해 Vectors 객체 만들어서 TEXT 필드로 전달
# 파일 내 첫번째 행은 임베딩 행렬 크기기 때문에 무시하게 된다.
vectors = Vectors(name='word2vec.pt')

# 생성한 vector 객체를 단어장을 만들면서 필드로 전달
TEXT.build_vocab(dataset, min_freq=3, vectors=vectors)

# 필드에 <unk>과 <pad> 토큰을 포함한 임베딩 행렬로 구성됨
print(TEXT.vocab.vectors.size())

# 임베딩 층을 필드에 내장된 임베딩 행렬로 초기화
# freeze 비활성해야 계속 학습 가능
embedding = nn.Embedding.from_pretrained(TEXT.vocab.vectors, freeze=False)
embedding(torch.LongTensor([2]))

100%|██████████████████████████████████████████████████████████████████████████| 30382/30382 [00:05<00:00, 5898.31it/s]


torch.Size([30369, 100])


tensor([[-6.2962e-02,  1.7045e-01,  6.8156e-02,  1.2706e-01, -1.1174e-01,
         -1.7718e-01,  5.2652e-02,  2.2250e-01, -4.3966e-02, -7.6643e-02,
          1.7943e-02, -8.5420e-02,  1.0086e-02,  2.1237e-01,  6.9371e-02,
         -3.0939e-02, -1.2076e-01, -7.4392e-02,  1.0391e-01, -1.2644e-01,
          1.4588e-01,  2.3058e-02,  8.4698e-02,  5.6798e-02,  9.5416e-02,
          9.2290e-03, -1.8434e-01,  5.8887e-02, -1.4722e-01,  2.7922e-02,
          6.0077e-02, -1.1426e-01,  5.2521e-02, -4.5747e-02, -1.3291e-01,
          1.2034e-01,  4.6845e-02, -2.4517e-05,  4.8735e-02,  1.7887e-02,
          1.9254e-02, -7.3278e-03, -1.8899e-01, -8.1516e-02, -4.3002e-02,
         -1.2576e-01, -1.0177e-01, -1.0308e-01,  2.1195e-01,  4.9919e-02,
          7.3218e-02,  7.2246e-03, -4.4953e-03,  4.6088e-02,  1.4022e-01,
         -1.9702e-02,  3.8092e-03, -6.4428e-02,  4.4049e-02,  1.0571e-01,
          1.5873e-02, -3.6440e-02,  3.0050e-02,  1.0486e-02, -6.0427e-02,
          1.3021e-01,  4.7938e-03,  7.

In [9]:
TEXT

In [10]:
dataset

In [11]:
vectors

In [12]:
TEXT.vocab

In [13]:
TEXT.vocab.vectors

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0630,  0.1705,  0.0682,  ..., -0.0908,  0.0241,  0.0041],
        ...,
        [-0.0825,  0.1206,  0.0799,  ..., -0.0981, -0.0391,  0.1410],
        [-0.1366,  0.0826,  0.0267,  ..., -0.1901, -0.0620,  0.1848],
        [-0.0333,  0.1729, -0.0055,  ..., -0.1404, -0.0786,  0.1420]])

In [14]:
TEXT.vocab.vectors.shape

torch.Size([30369, 100])